# This notebook runs all the different strategies with their dedicated notebook

Several design $\mathbb{X} = (\mathbf{x}_j)_{j=1}^n$ are investigated to ensure robustness. Each one is associated with a "seed_x", which is the random seed used to generate the design. For every design, a csv file called "results_measures_seed_x{seed_x}.csv" is stored, with the 10 first rows corresponding to the true values of the outputs and the 10 others to the noisy observations, obtained with sigma = 0.9 for the two first output variables and 0.3 for the last one. 

In [ ]:
import os, sys
import numpy                as np
import pandas as pd
import matplotlib.pyplot    as plt
import pandas as pd
from joblib import Parallel, delayed
from scipy.stats import multivariate_normal, truncnorm, norm
from scipy.optimize import minimize
import pickle
import random
from matplotlib.pyplot import figure
from scipy.stats import qmc, uniform
import matplotlib.patches as mpatches

from pymcmcstat.MCMC import MCMC
from pymcmcstat.ParallelMCMC import ParallelMCMC, load_parallel_simulation_results
from pymcmcstat.chain import ChainStatistics as CS
from pymcmcstat.chain import ChainProcessing as CP


In [ ]:
%run gp_simus.ipynb #notebook for the metamodel 
%run utils_calib.ipynb #notebook with needed function for the methods
%run utils_plot_errors.ipynb #notebook with needed function for the final plots


sigma = [0.9,0.9,0.3] #Standard deviation of the observation noise for each output variable
bMINlambda = np.array([330, 70,  8000, 3000, 0.1, 0.1]) #lower bounds for lambda 
bMAXlambda = np.array([470, 150, 10000, 5000, 5, 5.0]) #upper bounds for lambda

seed_x = 123456 #indicates which design $x_j$ to work with
suff = "" #suffix for potential smallnoise study
results_measures = pd.read_csv(f"measurement_points/results_measures_seed_x{seed_x}" + suff + ".csv", index_col=0)
true_values = results_measures.iloc[:10,:] #get true values
results_measures = results_measures.iloc[10:,:] #get noisy observations

with open(f'surrogate_models/mm_list_seed_x{seed_x}.pkl', 'rb') as f: #get the GP hyperparameters
    mm_list = pickle.load(f)
    
def save_results(data, file, pre_path, calib = None): #function to save the results obtained with each strategy
    if(os.path.isdir(pre_path)==False):  
        os.mkdir(pre_path)
    if(os.path.isdir(pre_path + f"/calib_{calib}")==False): 
        os.mkdir(pre_path + f"/calib_{calib}")
    data.to_csv(pre_path +f"/calib_{calib}/{file}")



calib_only = [1,2,3] #index of the calibration problems to consider. [1,2,3] means that the three configurations will be tested: observations of the first variable, of the second one, or of the third one. 

# Hierarchical model

In [ ]:
index_lambda_p = [2,3,4,5] #indexes of the four physical parameters
index_lambda_q = [0,1] #indexes of the two numerical parameters
model_error = True #Model error is considered, so hierarchical bayesian model for the two numerical parameters
scale = 0.45 #std deviation of the truncated gaussian prior
pre_path = f"seed_x{seed_x}" + suff + "/hierarchical_model" #where to store the results

In [ ]:
%run investigate_alphamap.ipynb

In [ ]:
%run bayes_alpha.ipynb

In [ ]:
%run bayes_lambda.ipynb 

In [ ]:
%run full_bayes.ipynb

# Uniform prior

In [ ]:
index_lambda_p = [2,3,4,5,0,1] #here, all variables are considered with uniform prior
index_lambda_q = [] #no hierarchical model
model_error = True 
scale = None 
pre_path = f"seed_x{seed_x}" + suff + "/uniform_error" #where to store the results

In [ ]:
%run bayes_lambda.ipynb

# No model error

In [ ]:
index_lambda_p = [2,3,4,5] #indexes of the four physical parameters
index_lambda_q = [0,1] #indexes of the two numerical parameters
model_error = False #No model error is considered, so the two numerical parameters are fixed
scale = None 
pre_path = f"seed_x{seed_x}" + suff + "/no_error"

In [ ]:
%run bayes_lambda.ipynb

# Embedded discrepancy

In [ ]:
pre_path = f"seed_x{seed_x}" + suff + "/embedded_discrepancy"

# %run embedded_discrepancy.ipynb